In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.mva.*
import jline.solvers.nc.*
import jline.solvers.qns.*
import jline.util.matrix.*

In [ ]:
val model = Network("model")

// Create nodes
val delay = Delay(model, "Delay")
val queue1 = Queue(model, "Queue1", SchedStrategy.FCFS)
queue1.setNumberOfServers(3)  // Multi-server queue with 3 servers

// Create two closed job classes
val jobclass1 = ClosedClass(model, "Class1", 4, delay, 0)  // 4 jobs of Class1
val jobclass2 = ClosedClass(model, "Class2", 2, delay, 0)  // 2 jobs of Class2

In [ ]:
// Set service processes for both classes
delay.setService(jobclass1, Exp(1.0))
delay.setService(jobclass2, Exp(1.0))

queue1.setService(jobclass1, Exp(1.0))   // Class1: slower service
queue1.setService(jobclass2, Exp(10.0))  // Class2: faster service

In [ ]:
// Set routing matrix - serial routing for both classes
val P = model.initRoutingMatrix()
val nodes = arrayOf(delay, queue1)

// Use serial routing for both classes (cyclic: Delay -> Queue1 -> Delay -> ...)
val serialRoute1 = Network.serialRouting(nodes)
val serialRoute2 = Network.serialRouting(nodes)

// Set routing for Class1
P.set(jobclass1, jobclass1, delay, delay, serialRoute1.get(0, 0))
P.set(jobclass1, jobclass1, delay, queue1, serialRoute1.get(0, 1))
P.set(jobclass1, jobclass1, queue1, delay, serialRoute1.get(1, 0))
P.set(jobclass1, jobclass1, queue1, queue1, serialRoute1.get(1, 1))

// Set routing for Class2
P.set(jobclass2, jobclass2, delay, delay, serialRoute2.get(0, 0))
P.set(jobclass2, jobclass2, delay, queue1, serialRoute2.get(0, 1))
P.set(jobclass2, jobclass2, queue1, delay, serialRoute2.get(1, 0))
P.set(jobclass2, jobclass2, queue1, queue1, serialRoute2.get(1, 1))

model.link(P)

In [ ]:
// Create array of different solvers to compare
val solvers = mutableListOf<NetworkSolver>()

solvers.add(CTMC(model))

// QNS solvers with different methods
solvers.add(QNS(model, "method", "conway"))
solvers.add(QNS(model, "method", "reiser"))
solvers.add(QNS(model, "method", "rolia"))
solvers.add(QNS(model, "method", "zhou"))

// MVA solvers with different multi-server approximations
solvers.add(MVA(model, "multiserver", "softmin"))
solvers.add(MVA(model, "multiserver", "seidmann"))

solvers.add(NC(model))

// Execute each solver and display results
for (solver in solvers) {
    try {
        val avgTable = solver.getAvgTable()
        avgTable.print()
    } catch (e: Exception) {
        println("  Error: ${e.message}")
    }
}

This multi-class repairmen model demonstrates:

1. **Multi-class System**: Two different job classes with different characteristics
2. **Multi-server Queue**: 3 servers at the repair station
3. **Service Differentiation**: Class1 (slower) vs Class2 (faster service)
4. **MVA Approximations**: Different methods for handling multi-server queues:
   - **softmin**: Soft minimum approximation
   - **seidmann**: Seidmann's approximation
5. **Resource Competition**: Both classes compete for the same servers

The model represents:
- Two types of machines requiring different repair times
- A repair shop with 3 technicians
- Different service priorities or complexities
- Comparison of analytical solution methods